In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.layers import (
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    Input,
    Lambda,
)
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.applications.mobilenet_v2 import (
    MobileNetV2,
    preprocess_input as mbv2_preprocess,
)

c:\Users\ADMIN\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
data_dir = "./data/train/"
labels_path = "./data/labels.csv"
img_size = (224, 224, 3)

In [4]:
df = pd.read_csv(labels_path)
df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [5]:
df["image_path"] = data_dir + df["id"] + ".jpg"
df = df.drop(columns="id")

In [6]:
breeds = sorted(df["breed"].unique().tolist())
len(breeds)

120

In [7]:
class_to_num = dict(zip(breeds, range(len(breeds))))

Put all images into 1 array n value [n, x1, x2, x3]: <br>
n: number of images<br>
x1: height of image<br>
x2: width of image<br>
x3: depth of image(1 for grayscale image, 3 for color image)


In [8]:
def image_processing(data: pd.DataFrame, image_size: tuple):
    X = np.zeros(
        [len(data), image_size[0], image_size[1], image_size[2]],
        dtype=np.uint8,
    )
    y = np.zeros([len(data), 1], dtype=np.uint8)
    for idx, row in tqdm(data.iterrows()):
        img_pixels = load_img(row["image_path"], target_size=image_size)
        X[idx] = img_pixels
        y[idx] = class_to_num[row["breed"]]
    return X, y

In [9]:
X, y = image_processing(df, img_size)

10222it [00:38, 266.24it/s]


Extract feature using MobileNetV2

In [10]:
def extract_feature(model, model_preprocess, image_size: tuple, data):
    input_layer = Input(image_size)
    processor = Lambda(model_preprocess)(input_layer)
    base_model = model(
        weights="imagenet", include_top=False, input_shape=image_size
    )(processor)
    maxPool = MaxPooling2D()(base_model)
    feature_extractor = Model(inputs=input_layer, outputs=maxPool)
    feature_maps = feature_extractor.predict(data, batch_size=64, verbose=1)
    return feature_maps

In [11]:
features = extract_feature(MobileNetV2, mbv2_preprocess, img_size, X)

160/160 [==============================] - 84s 520ms/step


In [12]:
le = LabelBinarizer()
y = le.fit_transform(y)

In [13]:
X_train, X_val, y_train, y_val = train_test_split(
    features, y, test_size=0.2, random_state=12
)

In [14]:
X_train = X_train / 255.0
X_val = X_val / 255.0

In [15]:
X_train.shape[1:]

(3, 3, 1280)

In [16]:
model = Sequential()
model.add(Input(X_train.shape[1:]))

model.add(Flatten())

model.add(Dropout(0.2))

model.add(Dense(128, activation="relu"))

model.add(Dense(len(breeds), activation="softmax"))
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 11520)             0         
                                                                 
 dropout (Dropout)           (None, 11520)             0         
                                                                 
 dense (Dense)               (None, 128)               1474688   
                                                                 
 dense_1 (Dense)             (None, 120)               15480     
                                                                 
Total params: 1,490,168
Trainable params: 1,490,168
Non-trainable params: 0
_________________________________________________________________


In [17]:
epochs = 20
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=64,
)

Epoch 1/20
128/128 [==============================] - 2s 10ms/step - loss: 4.1912 - accuracy: 0.2284 - val_loss: 3.1802 - val_accuracy: 0.4572
Epoch 2/20
128/128 [==============================] - 1s 9ms/step - loss: 2.2540 - accuracy: 0.5931 - val_loss: 1.6546 - val_accuracy: 0.6572
Epoch 3/20
128/128 [==============================] - 1s 9ms/step - loss: 1.3066 - accuracy: 0.7174 - val_loss: 1.2103 - val_accuracy: 0.6890
Epoch 4/20
128/128 [==============================] - 1s 10ms/step - loss: 0.9763 - accuracy: 0.7661 - val_loss: 1.0188 - val_accuracy: 0.7198
Epoch 5/20
128/128 [==============================] - 1s 9ms/step - loss: 0.7980 - accuracy: 0.8010 - val_loss: 0.9323 - val_accuracy: 0.7379
Epoch 6/20
128/128 [==============================] - 1s 10ms/step - loss: 0.6896 - accuracy: 0.8272 - val_loss: 0.8722 - val_accuracy: 0.7501
Epoch 7/20
128/128 [==============================] - 1s 9ms/step - loss: 0.6048 - accuracy: 0.8438 - val_loss: 0.8446 - val_accuracy: 0.7565
Epo

In [18]:
test_path = "./data/test/"
img_test_files = os.listdir(test_path)[:5]

In [22]:
for img in img_test_files:
    file_name = os.path.join(test_path, img)
    image_test = load_img(file_name, target_size=img_size)
    image_test = np.array(image_test)
    image_test = image_test.reshape((1, 224, 224, 3))
    image_test = extract_feature(
        MobileNetV2, mbv2_preprocess, img_size, image_test
    )
    breed_index = model.predict(image_test).argmax(axis=-1)
    breed_label = [
        label for label, index in class_to_num.items() if index == breed_index
    ][0]

    print(breed_label)

1/1 [==============================] - 0s 16ms/step
japanese_spaniel
1/1 [==============================] - 0s 14ms/step
samoyed
1/1 [==============================] - 0s 14ms/step
english_setter
1/1 [==============================] - 0s 15ms/step
pug
1/1 [==============================] - 0s 14ms/step
tibetan_terrier
